На этом занятии мы попробуем задачу регрессии. Данные в этой же папке, будем тренироваться на датасете фильмов с IMDB

Перед обучением обучением модели, нужно подготовить данные:

- найти\собрать данные
- почистить и предобработать
- преобразовать в матрицы 


In [1]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
data = pd.read_csv('IMDB-Movie-Data.csv')
print(data.shape)

data.head(3)

(1000, 12)


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


## Что делать с NaN?

In [3]:
# 3. Превратить NaN в средние значения по колонке

# вычисляем средние для колонок с пустыми значениями
meta_mean = data.Metascore.mean()
rev_mean = data['Revenue (Millions)'].mean()

#заменяем пустоты на средние значения
data.Metascore.fillna(meta_mean, inplace=True)
data['Revenue (Millions)'].fillna(rev_mean, inplace=True)

# проверяем присутствие NaN
data.isna().any()

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
dtype: bool

## Подготовка данных

Попробуем предсказывать рейтинг фильма по данным его описания, года, длины в минутах и кассовых сборов

Колонка "Rating" станет **целевой переменной, или таргетом** (y)<br>
Остальных данные будут **обучающей выборкой** (X)

In [4]:
data.Description

0      A group of intergalactic criminals are forced ...
1      Following clues to the origin of mankind, a te...
2      Three girls are kidnapped by a man with a diag...
3      In a city of humanoid animals, a hustling thea...
4      A secret government agency recruits some of th...
                             ...                        
995    A tight-knit team of rising investigators, alo...
996    Three American college students studying abroa...
997    Romantic sparks occur between two dance studen...
998    A pair of friends embark on a mission to reuni...
999    A stuffy businessman finds himself trapped ins...
Name: Description, Length: 1000, dtype: object

In [5]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/senya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/senya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [7]:
import re
from string import punctuation
def preprocess(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [lmtzr.lemmatize(token) for token in tokens if token]
    return tokens

In [8]:
# подготовим описания фильмов
data["text"] = data.Description.apply(preprocess) 

data["text"]

0      [a, group, of, intergalactic, criminal, are, f...
1      [following, clue, to, the, origin, of, mankind...
2      [three, girl, are, kidnapped, by, a, man, with...
3      [in, a, city, of, humanoid, animal, a, hustlin...
4      [a, secret, government, agency, recruit, some,...
                             ...                        
995    [a, tight-knit, team, of, rising, investigator...
996    [three, american, college, student, studying, ...
997    [romantic, spark, occur, between, two, dance, ...
998    [a, pair, of, friend, embark, on, a, mission, ...
999    [a, stuffy, businessman, find, himself, trappe...
Name: text, Length: 1000, dtype: object

In [9]:
data.text.values

array([list(['a', 'group', 'of', 'intergalactic', 'criminal', 'are', 'forced', 'to', 'work', 'together', 'to', 'stop', 'a', 'fanatical', 'warrior', 'from', 'taking', 'control', 'of', 'the', 'universe']),
       list(['following', 'clue', 'to', 'the', 'origin', 'of', 'mankind', 'a', 'team', 'find', 'a', 'structure', 'on', 'a', 'distant', 'moon', 'but', 'they', 'soon', 'realize', 'they', 'are', 'not', 'alone']),
       list(['three', 'girl', 'are', 'kidnapped', 'by', 'a', 'man', 'with', 'a', 'diagnosed', '23', 'distinct', 'personality', 'they', 'must', 'try', 'to', 'escape', 'before', 'the', 'apparent', 'emergence', 'of', 'a', 'frightful', 'new', '24th']),
       list(['in', 'a', 'city', 'of', 'humanoid', 'animal', 'a', 'hustling', 'theater', "impresario's", 'attempt', 'to', 'save', 'his', 'theater', 'with', 'a', 'singing', 'competition', 'becomes', 'grander', 'than', 'he', 'anticipates', 'even', 'a', 'it', 'finalist', 'find', 'that', 'their', 'life', 'will', 'never', 'be', 'the', 'same'

In [10]:
input_text = list(data.text.values)

In [11]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
documents[10:12]

[TaggedDocument(words=['the', 'adventure', 'of', 'writer', 'newt', 'scamander', 'in', 'new', "york's", 'secret', 'community', 'of', 'witch', 'and', 'wizard', 'seventy', 'year', 'before', 'harry', 'potter', 'read', 'his', 'book', 'in', 'school'], tags=[10]),
 TaggedDocument(words=['the', 'story', 'of', 'a', 'team', 'of', 'female', 'african-american', 'mathematician', 'who', 'served', 'a', 'vital', 'role', 'in', 'nasa', 'during', 'the', 'early', 'year', 'of', 'the', 'u.s', 'space', 'program'], tags=[11])]

обучаем модель на текстах описаний фильмов (можно поизменять параметры)

In [12]:
n = 7
model = Doc2Vec(documents, vector_size=n, window=2, min_count=1, workers=4, hs=1)

In [13]:
model.save("D2V.model") # сохранение модели

In [14]:
# так можно посмотреть на векторы текстов, на которых училась модель
# индекс [] около documents -- это индекс текста из датасета

model.dv[documents[0].tags[0]]


array([ 0.14720327, -0.03376976, -0.3993985 ,  0.44332057,  0.11461276,
        0.09021867,  0.05458991], dtype=float32)

Теперь нужно добавить векторы в датасет с остальными параметрами

In [15]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model.dv[x.tags][0])
    vectors.append(vec)

In [16]:
cols = [f'v{i}' for i in range(n)]
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=cols)

split_df


,v0,v1,v2,v3,v4,v5,v6
0,0.147203,-0.033770,-0.399399,0.443321,0.114613,0.090219,0.054590
1,-0.148316,0.082283,-0.375760,0.368563,0.264143,-0.206247,0.173018
2,0.272906,-0.306117,-0.158219,0.559273,-0.268186,0.089990,0.642864
3,0.049968,-0.001687,-0.152468,0.610013,-0.080624,-0.027086,0.650576
4,0.088828,-0.081679,-0.213298,0.653106,-0.144925,-0.164846,0.569208
...,...,...,...,...,...,...,...
995,-0.119215,0.173668,-0.566188,0.335503,-0.035567,0.249297,0.721935
996,0.466348,0.014694,0.034058,0.537794,-0.039908,-0.773337,0.381744
997,-0.029680,0.013548,-0.508374,0.693552,0.305587,-0.273351,0.151031
998,0.023137,0.045992,-0.016479,0.083811,0.109477,0.104356,0.051287


In [17]:
# теперь добавим его к основному датафрейму
result = data.join(split_df, how='left')
result.shape

(1000, 20)

In [18]:
result.head(3)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,text,v0,v1,v2,v3,v4,v5,v6
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,"[a, group, of, intergalactic, criminal, are, f...",0.147203,-0.033770,-0.399399,0.443321,0.114613,0.090219,0.054590
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,"[following, clue, to, the, origin, of, mankind...",-0.148316,0.082283,-0.375760,0.368563,0.264143,-0.206247,0.173018
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,"[three, girl, are, kidnapped, by, a, man, with...",0.272906,-0.306117,-0.158219,0.559273,-0.268186,0.089990,0.642864


In [19]:
# переопределим датасет, оставив только важное

data_sm = result[['Runtime (Minutes)',"Year",
                'Rating', 'Votes',
                'Revenue (Millions)','Metascore',*cols]
              ]


data_sm.head(3)

,Runtime (Minutes),Year,Rating,Votes,Revenue (Millions),Metascore,v0,v1,v2,v3,v4,v5,v6
0,121,2014,8.1,757074,333.13,76.0,0.147203,-0.033770,-0.399399,0.443321,0.114613,0.090219,0.054590
1,124,2012,7.0,485820,126.46,65.0,-0.148316,0.082283,-0.375760,0.368563,0.264143,-0.206247,0.173018
2,117,2016,7.3,157606,138.12,62.0,0.272906,-0.306117,-0.158219,0.559273,-0.268186,0.089990,0.642864


## Подготавливаем матрицы

In [20]:
# определяем X и y

X = data_sm.drop(["Rating"],axis=1).values 

display(X, X.shape)

array([[ 1.21000000e+02,  2.01400000e+03,  7.57074000e+05, ...,
         1.14612758e-01,  9.02186707e-02,  5.45899123e-02],
       [ 1.24000000e+02,  2.01200000e+03,  4.85820000e+05, ...,
         2.64143169e-01, -2.06247255e-01,  1.73018232e-01],
       [ 1.17000000e+02,  2.01600000e+03,  1.57606000e+05, ...,
        -2.68186420e-01,  8.99897367e-02,  6.42864287e-01],
       ...,
       [ 9.80000000e+01,  2.00800000e+03,  7.06990000e+04, ...,
         3.05587471e-01, -2.73351014e-01,  1.51031241e-01],
       [ 9.30000000e+01,  2.01400000e+03,  4.88100000e+03, ...,
         1.09476805e-01,  1.04355820e-01,  5.12865707e-02],
       [ 8.70000000e+01,  2.01600000e+03,  1.24350000e+04, ...,
        -3.27333570e-01,  2.20576167e-01,  2.17621133e-01]])

(1000, 12)

In [21]:
data_sm.isna().any()

Runtime (Minutes)     False
Year                  False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
v0                    False
v1                    False
v2                    False
v3                    False
v4                    False
v5                    False
v6                    False
dtype: bool

In [22]:
y = data_sm['Rating'].values # отдельно вынесли массив со значениями скорости ветра
y.shape

(1000,)

Иногда бывает полезно [нормализовать](https://en.wikipedia.org/wiki/Normalization_(statistics)) данные: это позволяет исправить ситуацию, когда признаки представлены в разных единацах измерения. 
Для этого используется StandardScaler. 

До нормализации:

In [23]:
list(X[0])

[121.0,
 2014.0,
 757074.0,
 333.13,
 76.0,
 0.147203266620636,
 -0.03376976028084755,
 -0.3993985056877136,
 0.4433205723762512,
 0.11461275815963745,
 0.0902186706662178,
 0.05458991229534149]

In [24]:
# использзуем стандартизатор
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)

После:

In [25]:
list(sc.fit_transform(X)[0])

[0.4163497512303056,
 0.37979525138136244,
 3.1126899627963738,
 2.5961363010556906,
 1.0233613578368184,
 -0.17279651117341935,
 0.444142925396914,
 -0.16466285153643292,
 -0.6109909839172423,
 1.180101564840596,
 0.4294406326562542,
 -1.389626396711949]

теперь с данными удобнее работать и обучать

In [26]:
from math import sqrt

In [27]:
alpha=0.0001
model = Ridge(alpha=alpha)
model.fit(X_train, y_train)

# давайте предскажем результат для тестовой выборки
y_preds = model.predict(X_test)
print('alpha', alpha)
print('MAE', round(mean_absolute_error(y_test, y_preds), 4))
print('MSE', round(mean_squared_error(y_test, y_preds), 4))
print('RMSE', round(sqrt(mean_squared_error(y_test, y_preds)), 4))

alpha 0.0001
MAE 0.4876
MSE 0.4664
RMSE 0.683


In [28]:
alpha = 2
model = Ridge(alpha=alpha)
model.fit(X_train, y_train)

# давайте предскажем результат для тестовой выборки
y_preds = model.predict(X_test)
print('alpha', alpha)
print('MAE', round(mean_absolute_error(y_test, y_preds), 4))
print('MSE', round(mean_squared_error(y_test, y_preds), 4))
print('RMSE', round(sqrt(mean_squared_error(y_test, y_preds)), 4))

alpha 2
MAE 0.4876
MSE 0.4666
RMSE 0.6831


In [29]:
alpha = 1000
model = Ridge(alpha=alpha)
model.fit(X_train, y_train)

# давайте предскажем результат для тестовой выборки
y_preds = model.predict(X_test)
print('alpha', alpha)
print('MAE', round(mean_absolute_error(y_test, y_preds), 4))
print('MSE', round(mean_squared_error(y_test, y_preds), 4))
print('RMSE', round(sqrt(mean_squared_error(y_test, y_preds)), 4))

alpha 1000
MAE 0.5625
MSE 0.5951
RMSE 0.7714


### оценка результатов алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*

Попробуйте разные значения для параметра регуляризации alpha при обучении модели. Как они влияют на величину ошибки?

Настройка параметра alpha для Ridge не особо влияет, если мы не берем экстремально большое значение alpha (тогда ошибка растет). 

Результаты для Ridge получились хорошие (mae < 0.5) при небольших альфа.

## Попробуем Лассо

In [30]:
alpha=0.001
model = Lasso(alpha=alpha)
model.fit(X_train, y_train)

# давайте предскажем результат для тестовой выборки
y_preds = model.predict(X_test)
print('alpha', alpha)
print('MAE', round(mean_absolute_error(y_test, y_preds), 4))
print('MSE', round(mean_squared_error(y_test, y_preds), 4))
print('RMSE', round(sqrt(mean_squared_error(y_test, y_preds)), 4))

alpha 0.001
MAE 0.4875
MSE 0.4666
RMSE 0.6831


Лассо выучился хорошо.

In [31]:
alpha = 1
model = Lasso(alpha=alpha)
model.fit(X_train, y_train)

# давайте предскажем результат для тестовой выборки
y_preds = model.predict(X_test)
print('alpha', alpha)
print('MAE', round(mean_absolute_error(y_test, y_preds), 4))
print('MSE', round(mean_squared_error(y_test, y_preds), 4))
print('RMSE', round(sqrt(mean_squared_error(y_test, y_preds)), 4))

alpha 1
MAE 0.7736
MSE 0.9734
RMSE 0.9866


Но Лассо более чувствителен к изменению alpha. Тут вышло плохо.

### LinearRegression

In [32]:
model = LinearRegression()
model.fit(X_train, y_train)

# давайте предскажем результат для тестовой выборки
y_preds = model.predict(X_test)

print('MAE', round(mean_absolute_error(y_test, y_preds), 4))
print('MSE', round(mean_squared_error(y_test, y_preds), 4))
print('RMSE', round(sqrt(mean_squared_error(y_test, y_preds)), 4))

MAE 0.4876
MSE 0.4664
RMSE 0.683


Больших различий между моделями нет.

О моделях и альфе (лямбде):


Чем больше лямбда, тем сильнее модель штрафуется за величину коэффициентов и их количество. Если занулить лямбду, мы получим самую обычную функцию потерь методом наименьших квадратов, соответственно — самую обычную регрессию.

Существенно отличие регрессии лассо от ридж в том, что лассо зануляет коэффициенты. То есть буквально перед какими-то фичами она ставит 0 и в модели они не рассматриваются. Ридж же может коэффициент сильно уменьшить, но не занулить. 


В общем, видимо, у нас не так много параметров и, возможно, данных, поэтому все хорошо справлется.